In [ ]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [33]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from googletrans import Translator

import ipywidgets as widgets
from IPython.display import display, clear_output

In [34]:
model = GPT2LMHeadModel.from_pretrained('saved_model_fico/')
tokenizer = GPT2Tokenizer.from_pretrained('saved_model_fico/')
translator = Translator()


In [35]:
def generate(entered_prompt):

  model.eval()
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  if entered_prompt.strip() == '':
    prompt = "<|startoftext|>"
  else:
    prompt = translator.translate(entered_prompt, src="sk").text

  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # print(generated)

  sample_output = model.generate(
                                  generated, 
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,   
                                  top_k=50, 
                                  max_length = 300,
                                  top_p=0.9, 
                                  num_return_sequences=1,
                                  pad_token_id=tokenizer.eos_token_id
                                  )

  return str(translator.translate(tokenizer.decode(sample_output[0], skip_special_tokens=True), dest="sk").text)

In [36]:
prompt = widgets.Text(placeholder='Náhodná téma',layout=widgets.Layout(width='200px', height='50px'))
text_0 = widgets.HTML(value="<h3>Zadaj tému na ktorú ti Robot Fico™ napíše status, ak nezadáš nič, téma bude náhodná.</h3>")


button_send = widgets.Button(
                description='Napíš status!',
                tooltip='Klikni na mňa',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='200px', height='50px')
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        response = generate(prompt.value)
        display(widgets.Image(value=open("StatusAI_fico.jpg", "rb").read(),format='jpg',width='500'))
        display(widgets.HTML(value=response))
        

button_send.on_click(on_button_clicked)

vbox = widgets.VBox([text_0, prompt, button_send, output])

In [37]:
# prompt = widgets.Text(placeholder='Náhodná téma',layout=widgets.Layout(width='200px', height='50px'))
# text_0 = widgets.HTML(value="<h3>Zadaj tému na ktorú ti AIrik Kaliňák™ napíše status, ak nezadáš nič, téma bude náhodná.</h3>")


# button_send = widgets.Button(
#                 description='Napíš status!',
#                 tooltip='Klikni na mňa',
#                 style={'description_width': 'initial'},
#                 layout=widgets.Layout(width='200px', height='50px')
#             )

# output = widgets.Output()

# def on_button_clicked(event):
#     with output:
#         clear_output()
#         response = generate(prompt.value)
#         display(widgets.Image(value=open("StatusAI_erik.jpg", "rb").read(),format='jpg',width='500'))
#         display(widgets.HTML(value=response))
        

# button_send.on_click(on_button_clicked)

# vbox = widgets.VBox([text_0, prompt, button_send, output])

In [38]:
page = widgets.HBox([vbox])
display(page)